Thomas Dougherty

Probability and Statistics for Computer Science


Introduction

    To Do:

    Figure out script to iterate through and cleanup all CSVs

    Store data from all CSVs

    Aggregate data from *all* CSVs into one dataframe
    

### March 2018

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper_functions import *

#read CSV for March 2018
services = pd.read_csv('data\\2018_03.csv')
#status column is not needed so that will be dropped
services.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services = services.astype({'date' : 'datetime64[ns]',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'})


weather = pd.read_csv('data\\weather.csv')
weather = helper.format_weather(weather,'March')
weather.head(10)
#services.head(10)

,STATION,LATITUDE,LONGITUDE,DATE,PRCP,SNOW
0,US1NJMC0040,40.292407,-74.706168,2018-03-01,0.00,0.0
1,US1NJMC0040,40.292407,-74.706168,2018-03-02,1.43,NaN
2,US1NJMC0040,40.292407,-74.706168,2018-03-03,0.72,NaN
3,US1NJMC0040,40.292407,-74.706168,2018-03-04,0.00,0.0
4,US1NJMC0040,40.292407,-74.706168,2018-03-05,0.00,0.0
5,US1NJMC0040,40.292407,-74.706168,2018-03-06,0.00,0.0
6,US1NJMC0040,40.292407,-74.706168,2018-03-08,NaN,NaN
7,US1NJMC0040,40.292407,-74.706168,2018-03-09,0.00,0.0
8,US1NJMC0040,40.292407,-74.706168,2018-03-10,0.00,0.0
9,US1NJMC0040,40.292407,-74.706168,2018-03-11,0.00,0.0


In [2]:
 #create new dataframe with data broken down by line
# function to create df. figure out how to import pandas into helper file???

njt_lines=services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services,njt_lines))

performance_by_line_march = helper.create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

performance_by_line_march.astype({'Longest Delay (minutes)' :'float16',
                            'Average Delay (minutes)' :'float16',
                            'Date of Longest Delay' : 'datetime64[ns]'
                            })

performance_by_line_march

,Longest Delay (minutes),Average Delay (minutes),Date of Longest Delay
Northeast Corrdr,91.062,4.253,2018-03-12
No Jersey Coast,100.250,3.932,2018-03-01
Main Line,86.000,3.542,2018-03-07
Morristown Line,100.312,3.781,2018-03-02
Gladstone Branch,99.062,3.654,2018-03-02
Raritan Valley,94.000,2.662,2018-03-07
Bergen Co. Line,79.000,2.964,2018-03-07
Atl. City Line,100.188,4.481,2018-03-07
Montclair-Boonton,111.188,3.382,2018-03-07
Princeton Shuttle,4.465,0.096,2018-03-04


### November 2018:

In [3]:
#read CSV for March 2018
services_nov = pd.read_csv('data\\2018_11.csv')
#status column is not needed so that will be dropped
services_nov.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services_nov.drop(services_nov[services_nov['type'] == 'Amtrak'].index, inplace=True)
services_nov.drop(services_nov[services_nov['line'] == 'Meadowlands Rail'].index, inplace=True)
services_nov.astype({'date' : 'datetime64[ns]',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'},
                errors='ignore')
services_nov.dtypes
services_nov

njt_lines=services_nov['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services_nov,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services_nov,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services_nov,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services_nov,njt_lines))

performance_by_line_nov = helper. create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

performance_by_line_nov.astype({'Longest Delay (minutes)' :'float16',
                            'Average Delay (minutes)' :'float16',
                            'Date of Longest Delay' : 'datetime64'
                            })

performance_by_line_nov

NameError: name 'create_new_dataframe' is not defined

In [ ]:
######### weather data 
#####
###
##
#weather = weather.astype({'DATE':'datetime64'})

#weather['DATE'].apply(pd.Timestamp.month_name)

#nov_weather = weather.loc[weather['DATE'].apply(pd.Timestamp.month_name) == 'November']
nov_weather.head(10)
